In [59]:
import json
import pandas as pd
import time
import gcsfs

In [60]:
from google.cloud import storage
from google.cloud import bigquery
from pathlib import Path
import os

In [76]:
PROJ_ROOT = Path().resolve().parent
KEYS_DIR = PROJ_ROOT / 'keys' 
keys = KEYS_DIR / 'key-query-export.json'

In [77]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = str(keys)

In [63]:
project = 'smart-science'
dataset_id = 'Learners_Questions'
table_id = 'Combined_tables_39k'
bucket_name = 'muse-data'

# Setup Storage vars
storage_client = storage.Client(project=project)
bucket = storage_client.get_bucket('muse-data')

# Setup BigQuery vars
bq_client = bigquery.Client()
dataset_ref = bq_client.dataset(dataset_id, project=project)
table_ref = dataset_ref.table(table_id)

#destination
destination_uri = "gs://{}/{}".format(bucket_name, "combined_tables_raw.json")


Forbidden: 403 GET https://www.googleapis.com/storage/v1/b/muse-data?projection=noAcl: big-query-admin@smart-science.iam.gserviceaccount.com does not have storage.buckets.get access to muse-data.

In [78]:
query_job_config = bigquery.QueryJobConfig()
query_job_config.write_disposition = 'WRITE_TRUNCATE'
# Set the destination table
original_table_ref = bq_client.dataset(dataset_id).table('New_table')
query_job_config.destination = table_ref

sql = """
    SELECT a.JiveLearner.id as learner, b.message.jive_id as question, a.answer, a.last_modified, a.JiveEnvelope.name, b.message.learner_name, a.tags.tag, a.tags.metatag, c.who, c.text, c.answer_type
    FROM `socos-quickstep-601.museathome.BinaryQuestionAnswer` as a
    JOIN `socos-quickstep-601.museathome.JiveEnvelope` as b ON a.JiveEnvelope.name=b.__key__.name
    JOIN `socos-quickstep-601.museathome.questions` as c ON CAST(b.message.jive_id AS NUMERIC) = c.id
    ORDER BY a.JiveLearner.id, b.message.learner_name, question ASC
"""

# Start the query, passing in the extra configuration.
query_job = bq_client.query(
    sql,
    # Location must match that of the dataset(s) referenced in the query
    # and of the destination table.
    location='US',
    job_config=query_job_config)  # API request - starts the query

query_job.result()  # Waits for the query to finish
print('Query results loaded to table {}'.format(table_ref.path))

Forbidden: 403 Access Denied: Table socos-quickstep-601:museathome.BinaryQuestionAnswer: User does not have permission to query table socos-quickstep-601:museathome.BinaryQuestionAnswer.

(job ID: 6c3ff9ea-e915-425a-9ad3-f0d1dd5cbf4f)

                                                                                      -----Query Job SQL Follows-----                                                                                       

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:    SELECT a.JiveLearner.id as learner, b.message.jive_id as question, a.answer, a.last_modified, a.JiveEnvelope.name, b.message.learner_name, a.tags.tag, a.tags.metatag, c.who, c.text, c.answer_type
   3:    FROM `socos-quickstep-601.museathome.BinaryQuestionAnswer` as a
   4:    JOIN `socos-quickstep-601.museathome.JiveEnvelope` as b ON a.JiveEnve

In [40]:
#set job configuration to extract job as JSON files
extract_job_config = bigquery.ExtractJobConfig()
extract_job_config.destination_format = 'NEWLINE_DELIMITED_JSON'
extract_job_config.write_disposition = 'WRITE_TRUNCATE'

In [41]:
extract_job = bq_client.extract_table(
    table_ref,
    destination_uri,
    # Location must match that of the source table.
    location="US",
    job_config=extract_job_config
)  # API request
extract_job.result()  # Waits for job to complete.

print(
    "Exported {}:{}.{} to {}".format(project, dataset_id, table_id, destination_uri)
)

Exported smart-science:Learners_Questions.Combined_Tables to gs://muse-data/combined_tables_raw.json


In [47]:
df = pd.read_json(destination_uri, lines=True)

In [48]:
df

,answer,answer_type,last_modified,learner,learner_name,metatag,name,question,tag,text,who
0,NaN,binary,2017-06-04 15:56:53.69455 UTC,4535845151309824,Carolina,[answer],UFqE8BiENzhk07YcA0oD8Eaxrb50LSts,368,[yes],Do you have reliable childcare for {kid_name}?,parent
1,NaN,binary,2017-06-04 15:56:57.46436 UTC,4535845151309824,Carolina,[answer],kLHlljhhWAnDEAixy2iaR9QkNhPYgdiw,371,[yes],Do you work outside of the home?,parent
2,NaN,binary,2017-08-31 11:44:55.22537 UTC,4535845151309824,Carolina,[answer],AI9iuqtEMakSt0EqPasH4B9MxQuDgice,38,[definitely],Did you graduate from high school?,parent
3,1.0,binary,2017-01-15 01:09:13.02664 UTC,4536217521618944,Katie,[answer],zUXTth86bHSuv5tzk3HZP3pyt2cj7Zhp,22,[yes],Do you know where the nearest playground is fr...,parent
4,1.0,binary,2017-01-15 01:11:47.43045 UTC,4536217521618944,Katie,[answer],cjTk7pkKA37b4TAOuDl1gO5n0JUxOW4f,22,[yes],Do you know where the nearest playground is fr...,parent
5,1.0,scale,2017-01-15 01:09:10.63448 UTC,4536217521618944,Katie,[answer],fF1jcJpeFa0QNrMZDLDDYfvHT3iT5OeD,236,[yes],"Does {kid_name} complete familiar books, espec...",child
6,1.0,scale,2017-01-15 01:11:49.84119 UTC,4536217521618944,Katie,[answer],iFDQCB2EFajl0ZOJRWxVFb2Stg7trWHb,283,[yes],Does {kid_name} enjoy trying new things?,child
7,1.0,scale,2017-01-15 01:08:00.33935 UTC,4536217521618944,Katie,[answer],qiLwtrCvLnM6nDyuHMyV0SieyuYxFtlq,285,[yes],Does {kid_name} enjoy creating elaborate prete...,child
8,0.0,binary,2017-01-15 01:13:38.63336 UTC,4536217521618944,Katie,[answer],P5apuN9tKCPat5dhGPvWCz6WIEeofC0s,41,[no],Are you the first person in your family to gra...,parent
9,1.0,scale,2017-01-15 01:07:52.71889 UTC,4536217521618944,Katie,[answer],wlbqIQAZcupMEcczJxbCGwX2QiNKJogQ,434,[yes],Does {kid_name} usually share with others?,child


In [58]:
df['metatag'].dtype

dtype('O')

In [68]:
df.iloc[35, 5]

['feedback', 'answer']

In [72]:
count = 0
for i in df.index:
    if 'feedback' in df.iloc[i, 5]:
        count = count + 1
print(count)

15110


In [73]:
count = 0
for i in df.index:
    if 'answer' in df.iloc[i, 5]:
        count = count + 1
print(count)

30929
